In [38]:
from scipy.optimize import linprog
import numpy as np

# Кейс ФОРЕВА №1

## Исходные данные

In [39]:
price = np.array([180, 285, 210])
rm_costs = np.array([30+35, 35+30, 75])

demand = [40,80,40]

resource_plan = np.array([
    [0,6+28,14],
    [4+5+15,4+5,15],
    [15+18, 9+6, 12+10],
    [20, 18, 20+7],
    [8,8+9,0]
])

resource_count = np.array([1,2,2,2,1])
resource_downtime = np.array([15,120,60,20,0])
downtime_count = np.array([3, 4, 5, 4, 2])
time_limit = 8 * 5 * 60


## Рассчеты
Заставляем машинку складывать нули и единицы

In [40]:
profit = np.subtract(price,rm_costs)
resource_limits = np.multiply(resource_count, time_limit)
resource_downtime_total = np.multiply(resource_downtime,downtime_count)
resource_usage = np.sum(resource_plan, axis=1)

c = np.negative(profit)
A_ub = resource_plan
downtime_total =  np.multiply(resource_downtime,downtime_count)
b_ub = np.subtract(np.multiply(resource_count,time_limit),downtime_total)
bounds = [(0, x) for x in demand]

result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='simplex')

## Результаты рассчетов

In [41]:
print(f'Оптимизированная прибыль: {-result.fun-16000}')
print(f'Производственный план:')
j = 0
for i in result.x:
    j+=1
    print(f'Продукт №{j}: {i}')


Оптимизированная прибыль: 5614.705882352944
Производственный план:
Продукт №1: 40.0
Продукт №2: 52.794117647058826
Продукт №3: 40.0
